# Introduction to Low-Rank Approximation

---

> Author: <font color='#f78c40'>Samuel Farrens</font>    
> Year: 2017  
> Email: [samuel.farrens@gmail.com](mailto:samuel.farrens@gmail.com)  
> Website: <a href="https://sfarrens.github.io" target="_blank">https://sfarrens.github.io</a>

---


## Contents

---
 
1. [Set-Up](#Set-Up)
1. [Introduction](#Introduction)
1. [Hints](#Hints)
1. [Exercise Solutions](#Exercise-Solutions)

---

## Set-Up

Here we will import a couple of packages that we will need throughout the notebook.

In [1]:
# Tell Jupyter to display plots in this notebook.
%matplotlib inline

# Import the numpy package with the alias np.
import numpy as np           

# Import the pyplot package from matplotlib with the alias plt.
import matplotlib.pyplot as plt  
from matplotlib import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

# Ignore warnings.
import warnings
warnings.filterwarnings("ignore")

---

## Introduction



---

## Rank of a Matrix

The rank of a matrix can be defined in the following ways:

1. the maximum number of linearly independent column vectors in a given matrix
1. the maximum number of linearly independent row vectors in a given matrix

Both of these definitions are equivalent.

For example if

$$ A = \begin{bmatrix} 6 & 1 & 2 \\ 5 & 6 & 1 \\ 5 & 7 & 7 \end{bmatrix} $$

then

$$\textrm{rank}(A) = 3$$

since all three rows are linearly idependent. $A$ is then said to have **full rank**.

This can be implemented in Python as follows

In [2]:
A = np.array([[6, 1, 2], [5, 6, 1], [5, 7, 7]])

print 'A ='
print A
print ''
print 'rank(A) =', np.linalg.matrix_rank(A)

A =
[[6 1 2]
 [5 6 1]
 [5 7 7]]

rank(A) = 3


If, however,

$$ A = \begin{bmatrix} 1 & 0 & 1 \\ -2 & -3 & 1 \\ 3 & 3 & 0 \end{bmatrix} $$

then

$$\textrm{rank}(A) = 2$$

since the third row is simply the difference of the first two, which are linearly idependent. In this case $A$ is **rank deficient** or has **reduced rank**.

In [3]:
A = np.array([[1, 0, 1], [-2, -3, 1], [3, 3, 0]])

print 'A ='
print A
print ''
print 'rank(A) =', np.linalg.matrix_rank(A)

A =
[[ 1  0  1]
 [-2 -3  1]
 [ 3  3  0]]

rank(A) = 2
